# How to run the pipeline on a set of imager data and create and subtract a sky background

If you have a set of imager data without extended sources such as a nebula or extended galaxy structure, you can create a mean (or median) background image from the data itself and subtract it off after the level two imaging pipeline stage is run to remove any remaining structure from imperfect flat fields or other issues. MIRI images have higher backgrounds at longer wavelengths, so this will be more useful for longer wavelength images, but can be used for any filter, so long as there is no extended source. 

This example notebook will demonstrate how to process a data set through the stage one and two imager pipelines, then use the cal.fits files to create the background sky and subtract it from each of the individual science images. Once you have the background subtracted images, the different dithers are then combined into a single image in the level 3 imaging pipeline.

The code used for the creation and subtraction of the background sky image was created by Karl Gordon and can be found separately on this github page: https://github.com/STScI-MIRI/Imaging_ExampleNB

The pipeline documentation can be found here: https://jwst-pipeline.readthedocs.io/en/latest/

The pipeline code and install directions are available on GitHub: https://github.com/spacetelescope/jwst

The steps in this notebook are:


  1)  Read in list of uncalibrated data (uncal.fits) files.

  2)  Process through calwebb_detector1.

  3)  Process ramp fit (rate.fits) files through calwebb_image2.
    
  4)  Create and remove a mean sky background from each calibrated (cal.fits) image.

  5)  Create an assocation file for the calibrated files.

  6)  Run the calibrated files through calwebb_image3 using the association file.
    
Setup needed before getting started.

* Place notebook and data into the directory where you wish to process your data.

* Install pipeline (and pip install any missing modules you encounter when you try to run).


### Caveats: This method of creating a background to subtract from the data should only be used in fields without extended sources such as galaxy or nebula emission that stretches across the image. It also will not work well with only a couple of dithers. The more dithers you have, the better this will work. Please examine your data before and after, and examine the output sky image to determine whether this does or does not work for your data specifically.

This notebook was written in July 2023, and was meant to work with pipeline version 1.11.1. Future versions of the pipeline should work, but backward compatibility of notebooks is not guaranteed with all python package updates.

## Information on Running the pipeline

This notebook utilizes the .call() method of running the notebook in python. With this method, the pipeline will retrieve and use any parameter reference file that applies to your data. These files will set certain parts of the code to run or be skipped, or select any parameters that the instrument teams have put as preferred defaults for most science cases. If you run with this method and do not set any custom parameters, you will replicate what you would get from the automated pipeline and populated in MAST. 

If you wish to customize the parameters, you can read the documentation in the ReadTheDocs link posted above. Each step in the pipeline has a section in the documentation that explains the parameters relevant to that step.  To set the parameters, a configuration dictionary is set up (and shown in the notebook as examples), then that custom configuration overrides the default values that would have come from the parameter reference files.

Most parameters that would need customization come in the stage three pipeline, but the jump step in the first pipeline stage, calwebb_detector1, also has a more extensive set of customizable parameters, and reading the documentation for this step could prove useful.

In order to run the pipeline and retrieve the necessary reference files from CRDS, there are a couple of CRDS variables that need to be set. CRDS_PATH tells the pipeline where to save and retrieve reference files. This should be set to a local directory where new reference files will be downloaded, and frequently used reference files can be retrieved and used without needing to pull large files across your internet connection. The other path that needs to be set is the CRDS_SERVER_URL, to know where to look to retrieve any new reference files. That path should be set to https://jwst-crds-stsci.edu. In this notebook, these are set using the os.environ statements below.

NOTE: you should store only CRDS reference files in your CRDS cache directory. This is in case you need to delete and redo your CRDS cache area at any point, and you don't want to chance erasing anything that should not be deleted.

os.environ['CRDS_PATH'] = '/local_crds_path/crds_cache_ops/'

os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds-stsci.edu'

### Using datamodels

This notebook will show how to use datamodels in working with your data. 
The basic steps involve reading a file into a datamodel type and then opening the individual named extensions as needed.

Header data is stored in the .meta extension.

Science data is stored in the .data extension.

Data quality flags can be accessed with .groupdq (in RampModel data files that allow accessing dq flags of each group in the data) or .dq (in ImageModel data files after calwebb_detector1 is run).

For more in-depth information on datamodels and how to use them: https://stdatamodels.readthedocs.io/en/latest/jwst/datamodels/index.html#data-models

### Import statements
Import modules that will be needed to read in data, run the pipeline and display any plots or visualizations.

Packages to be sure are installed:
* jwst
* regions
* jupyter

The CRDS path information needs to be set before importing any crds or jwst packages. The jwst installation instructions here: https://jwst-pipeline.readthedocs.io/en/latest/getting_started/quickstart.html show how to set up the CRDS information in a setup file (such as bash.profile or other setup file), but if the paths are not set in a setup file, they can be set in a notebook using os.environ as shown below.

In [ ]:
# Changes for the Science Platform environment
# Preparing cached data
import os
import glob

preloaded_fits_dir = "/home/shared/preloaded-fits/jwebbinar_31/miri/"
for filename in glob.glob(os.path.join(preloaded_fits_dir, "*.fits")):
    basename = os.path.basename(filename)
    if not os.path.exists(basename):
        os.symlink(filename, basename)
        
# Installing regions from pypi
!pip install regions

In [ ]:
# Set CRDS path info
import os

os.environ['CRDS_PATH'] = os.environ['HOME']+'/crds_cache/' 
os.environ['CRDS_SERVER_URL'] = 'https://jwst-crds.stsci.edu'

print('CRDS cache location: {}'.format(os.environ['CRDS_PATH']))

In [ ]:
#import jwst pipeline modules
import jwst
from jwst.pipeline import Detector1Pipeline, Image2Pipeline, Image3Pipeline # pipeline modules
from jwst import datamodels
from jwst.datamodels import RampModel, ImageModel, dqflags # Data models and dq values

# Needed for associations
from jwst import associations
from jwst.associations.lib.rules_level3_base import DMS_Level3_Base
from jwst.associations import asn_from_list

# Other modules/functions to work with and examine data
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import glob

from astropy.io import ascii, fits
from astropy.visualization import simple_norm
from astropy.modeling import models, fitting

from astropy.stats import sigma_clipped_stats
from astropy.convolution import Gaussian1DKernel, convolve
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord

from regions import Regions
from tweakwcs import JWSTgWCS

# Box download imports 
from astropy.utils.data import download_file
from pathlib import Path
from shutil import move
from os.path import splitext

import crds

In [ ]:
# Print the version of the JWST pipeline.

print(jwst.__version__)

### Load helper scripts for sky creation and subtraction

Load scripts to create and subtract the mean sky background image, as well as those for displaying images.

In [ ]:
# Parameters to set for the make_sky module

# allow for varying background
scalebkg = False

# experimental options for additional exclusion
exclude_above = None  # exclude all regions above the provided value in MJy/sr
exclude_delta = None  # exclude all regions above bkg+exclude_delta given in MJy/sr
ds9region = None  # provide a ds9 region file of regions to exclude

# display range for images - just for image display
drange_cal = [4., 10.]
drange_ssub = [-0.1, 5.0]
dmap = "afmhot"  # same as ds9 bb


# Parameter needed for calwebb_image3 output
rotation = None

In [ ]:
# Code used to create the background image and subtract it from all input cal images
# The combination method can be changed between mean or median further down in this code 


def make_sky(
    files,
    subfiles=None,
    scalebkg=False,
    exclude_above=None,
    exclude_delta=None,
    ds9regions=None,
):
    """
    Make sky background by sigma clipping in image coordinates and subtract it
    from all the input files.
    Parameters
    ----------
    files : strs
       Array of cal files to use to create the sky image
    subfiles : str
       Array of files to subtract sky from. If None, sky will be subracted from all input images. [default=None]
    scalebkg : boolean
       Scale each image by its median to the average value [default=False]
    exclude_above : float
       Exclude data above this value from the sky creation
    exclude_delta : float
       Exclude data above the median bkg + this value from sky creation
    ds9regions : ds9 region file
       Exclude pixels inside ds9 regions from sky creation
    """
    if ds9regions is not None:
        ereg = Regions.read(ds9regions, format="ds9")
        # for creg in ereg:
        #     creg.radius *= 0.5

    istack = None
    for k, cfile in enumerate(files):
        print(f"processing {cfile}")
        cdata = datamodels.open(cfile)
        if istack is None:
            isize = cdata.data.shape
            istack = np.empty((isize[0], isize[1], len(files)))
            istackmed = np.empty((len(files)))
        tdata = cdata.data

        # remove all the non imager data
        # bdata = cdata.dq & dqflags.pixel["DO_NOT_USE"] > 0
        # tdata[bdata] = np.NaN

        if exclude_above is not None:
            tdata[tdata > exclude_above] = np.NaN

        if ds9regions is not None:
   

            fits_header, fits_hdulist = cdata.meta.wcs.to_fits()
            cwcs = WCS(fits_header)  # <-- "astropy" wcs

            pixx = np.arange(isize[1])
            pixy = np.arange(isize[0])
            imagex, imagey = np.meshgrid(pixx, pixy)
            imagera, imagedec = cwcs.wcs_pix2world(imagex, imagey, 0)
            # imagera, imagedec = cwcs.pixel_to_world(imagex, imagey, 0)
            skycoord = SkyCoord(imagera, imagedec, unit="deg")
            for creg in ereg:
                inoutimage = creg.contains(skycoord, cwcs)
                tdata[inoutimage] = np.NaN
            cdata.data = tdata
            cdata.write(cfile.replace("cal.fits", "cal_mask.fits"))
            # fits.writeto("test.fits", inoutimage * 1., overwrite=True)

        istackmed[k] = np.nanmedian(tdata)
        print(f"median sky = {istackmed[k]}")

        if exclude_delta is not None:
            tdata[tdata > istackmed[k] + exclude_delta] = np.NaN

        istack[:, :, k] = tdata

    # adjust the levels to the median
    # allows for data taken at different times with different backgrounds
    
    ##############################################
    
    #medsky = np.mean(istackmed)  ########### This is where the combination can be changed between mean or median
    medsky = np.median(istackmed)
    
    ##############################################
    
    if scalebkg:
        for k in range(len(files)):
            istack[:, :, k] += medsky - istackmed[k]
            print(k, np.nanmedian(istack[:, :, k]))
    else:
        print("Not scaling individual images to median bkg")

    skyflat_mean, skyflat_median, skyflat_std = sigma_clipped_stats(
        istack, sigma_lower=3, sigma_upper=1, axis=2
    )

    # subtract the sky properly adjusted from the data
    if subfiles is None:
        subfiles = files
    for k, cfile in enumerate(subfiles):
        cdata = datamodels.open(cfile)
        cdata.data -= skyflat_mean
        if scalebkg:
            print(cfile, medsky - istackmed[k])
            cdata.data += medsky - istackmed[k]
        else:
            print(cfile)
        ndata = np.isnan(cdata.data)
        #cdata.data[ndata] = 0.0  # This sets all NaNs to 0
        cdata.dq[ndata] = cdata.dq[ndata] & dqflags.pixel["DO_NOT_USE"]
        cdata.write(cfile.replace("_cal.fits", "_skysub_cal.fits")) # Set to either cal or i2d images

    return skyflat_mean


In [ ]:
# Helper script to plot images

from astropy.visualization import ImageNormalize, ManualInterval, SqrtStretch


def show_image(
    data_2d, vmin, vmax, xpixel=None, ypixel=None, title=None, dmap="binary",
):
    """Function to generate a 2D, log-scaled image of the data,
    with an option to highlight a specific pixel (with a red dot).

    Parameters
    ----------
    data_2d : numpy.ndarray
        Image to be displayed

    vmin : float
        Minimum signal value to use for scaling

    vmax : float
        Maximum signal value to use for scaling

    xpixel : int
        X-coordinate of pixel to highlight

    ypixel : int
        Y-coordinate of pixel to highlight

    title : str
        String to use for the plot title
    """
    norm = ImageNormalize(
        data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax), stretch=SqrtStretch()
    )
    fig, ax = plt.subplots(figsize=(12, 10))
    im = ax.imshow(data_2d, origin="lower", norm=norm, cmap=plt.get_cmap(dmap))

    if xpixel and ypixel:
        plt.plot(xpixel, ypixel, marker="o", color="red", label="Selected Pixel")

    fig.colorbar(im, label="DN")
    plt.xlabel("Pixel column")
    plt.ylabel("Pixel row")
    if title:
        plt.title(title)


In [ ]:
# Helper script to plot an image and overlay catalog sources

def overlay_catalog(
    data_2d,
    catalog,
    flux_limit=0,
    vmin=0,
    vmax=10,
    title=None,
    units="MJy/str",
    dmap="binary",
):
    """Function to generate a 2D image of the data,
    with sources overlaid.

    data_2d : numpy.ndarray
        2D image to be displayed

    catalog : astropy.table.Table
        Table of sources

    flux_limit : float
        Minimum signal threshold to overplot sources from catalog.
        Sources below this limit will not be shown on the image.

    vmin : float
        Minimum signal value to use for scaling

    vmax : float
        Maximum signal value to use for scaling

    title : str
        String to use for the plot title

    units : str
        Units of the data. Used for the annotation in the
        color bar
    """
    norm = ImageNormalize(
        data_2d, interval=ManualInterval(vmin=vmin, vmax=vmax), stretch=SqrtStretch()
    )
    fig, ax = plt.subplots(figsize=(12, 10))
    im = ax.imshow(data_2d, origin="lower", norm=norm, cmap=plt.get_cmap(dmap))

    for row in catalog:
        if row["aper_total_flux"].value > flux_limit:
            plt.plot(
                row["xcentroid"],
                row["ycentroid"],
                marker="o",
                markersize="3",
                color="red",
            )

    plt.xlabel("Pixel column")
    plt.ylabel("Pixel row")

    fig.colorbar(im, label=units)
    fig.tight_layout()
    plt.subplots_adjust(left=0.15)

    if title:
        plt.title(title)

### Read in data

For this notebook, the data is stored in Box and will be loaded into the working directory.
You can also just put the notebook and data into a single directory, or include path information as you work with the data. 



In [ ]:
# Read in dataset from Box

# def get_box_files(file_list):
#     for box_url,file_name in file_list:
#         if 'https' not in box_url:
#             box_url = 'https://stsci.box.com/shared/static/' + box_url
#         downloaded_file = download_file(box_url, timeout=600)
#         if Path(file_name).suffix == '':
#             ext = splitext(box_url)[1]
#             file_name += ext
#         move(downloaded_file, file_name)


# # F2100W data from a calibration program, target BD+60-1753 
# file_urls = ['https://stsci.box.com/shared/static/k0xukhkin1rpaczva8cdws0yrs478zvj.fits',
#              'https://stsci.box.com/shared/static/j9wd92jcjn2fqd7waptcyfnl9nks1an0.fits',
#              'https://stsci.box.com/shared/static/jw2aklrwwxsbzqfghpp57yuvscj6de08.fits',
#              'https://stsci.box.com/shared/static/9zetqwlbq3jtp4fv7zpl84eff0w2st37.fits',
#              'https://stsci.box.com/shared/static/lw4g22c1pxwfo3h9cw49y88tlc8m2i9b.fits']


# files = ['jw01027004001_02105_00001_mirimage_uncal.fits',
#          'jw01027004001_02105_00002_mirimage_uncal.fits',
#          'jw01027004001_02105_00003_mirimage_uncal.fits',
#          'jw01027004001_02105_00004_mirimage_uncal.fits',
#          'jw01027-o004_t006_miri_f2100w_i2d.fits']


In [ ]:
# # Run commands to load the data from box into current directory

# box_download_list = [(url,name) for url,name in zip(file_urls,files)]


# get_box_files(box_download_list)

# print(files)

In [ ]:
# If files area already in the directory with the data, and you only need the list of files for processing, uncomment these lines.

files = ['jw01027004001_02105_00001_mirimage_uncal.fits',
        'jw01027004001_02105_00002_mirimage_uncal.fits',
        'jw01027004001_02105_00003_mirimage_uncal.fits',
        'jw01027004001_02105_00004_mirimage_uncal.fits',
        'jw01027-o004_t006_miri_f2100w_i2d.fits']

In [ ]:
uncalfiles = files[:-1]
print(uncalfiles)  # print list of uncal files only, minus the i2d file

In [ ]:
# To just gather all uncalfiles in a specific folder: Do not do this if you have multiple datasets in a single folder

#uncalfiles = glob.glob('*uncal.fits')
#print(uncalfiles)

### Look at your data and header parameters

Read your data files into a RampModel data model to examine some header parameters.
Read a sample data file into a model to display the last frame in the ramp to look at the scene in the data.

In [ ]:
# Look at some header parameters from your data

print('File name, Instrument, Subarray,  Filter,  Nints,  Ngroups')

for file in uncalfiles: 

    imfile = RampModel(file) # Read files into datamodel

    header = imfile.meta # Read the meta data into a variable called 'header'
    
    # Read in the header keywords
    name = header.filename
    inst = header.instrument.name
    subarray = header.subarray.name
    filt = header.instrument.filter
    nints = header.exposure.nints
    ngroups = header.exposure.ngroups

    print(name, inst, subarray, filt, nints, ngroups)

In [ ]:
# If you need to find a specific header value to view, and you know the FITS header keyword, you 
# can use this code to find the datamodel equivalent of the keyword.

imfile.find_fits_keyword('DATE-OBS')


In [ ]:
# Read sample data file into a datamodel
uncal_im = RampModel(uncalfiles[0])

In [ ]:
# Look at the last frame of one of the data files to visualize the field of view

plt.figure(figsize=(20,20))

plt.imshow(uncal_im.data[0, -1, :, :], cmap='Greys', origin='lower', vmin=20000,vmax=40000)
plt.colorbar()
plt.show()

## Run calwebb_detector1 on your data

Set a few parameters for the jump step (most steps are fine to run with defaults).
Loop through and run all files through calwebb_detector1 to get 'rate.fits' files that contain slope fit images.

When setting up the pipeline stage to run, using the .call() method requires setting up dictionaries for each of the steps that will have parameters set that are different from the defaults.

#### The detector1 pipeline will take the longest amount of runtime in this notebook. If you do not wish to process all of your data over from the start, you can download the cal.fits files from the archive, load those into the notebook, and run the background creation code from that point.

In [ ]:
# Set a few Jump step parameters, for versions 1.10.0 and later (build version 9.2)
# find_showers turns the cr shower code on for MIRI. Set to False to skip this code. (Default is currently False.)
# Use mostly code defaults for the rest of the parameters.

rej_thresh = 5

expand_large_events = False  # This parameter is used for NIR instruments
find_showers = False  # This parameter is used for MIRI only

In [ ]:
# Set a few parameters and run each file through the Detector1 Pipeline

for file in uncalfiles:
    pipe = Detector1Pipeline()
    
    # set up output file name
    base, remainder = file.split('_uncal')
    outname = base
    
    # Set up dictionaries for step parameters
    cfg = dict()
    cfg['jump'] = {}
    cfg['jump']['save_results'] = True # The jump output file is useful if you want to track which frames have jumps flagged
    cfg['jump']['rejection_threshold'] = rej_thresh
    cfg['jump']['expand_large_events'] = expand_large_events
    cfg['jump']['find_showers'] = find_showers
    cfg['jump']['output_file'] = base + '.fits'
    #cfg['jump']['output_dir'] = datadir

    #cfg['ramp_fit'] ={}
    #cfg['ramp_fit']['output_file'] = base + '.fits'
    #cfg['ramp_fit']['output_dir'] = datadir


    pipe.call(file, steps=cfg, save_results=True, output_file =  base + '.fits')

#### Look at output rate image

Plot one of the output rate images to see what the image looks like. The code here will also mark any pixels with a DQ value of DO_NOT_USE in blue. These blue pixels are considered bad in some way and will not contribute to the final combined image.

In [ ]:
# Load and look at a single rate image
  
ratefiles = [ele.replace('uncal', 'rate') for ele in uncalfiles]
rate_im = ImageModel(ratefiles[0])

In [ ]:
# Look at the averaged slope image, and plot the DO_NOT_USE dq flagged pixels in blue

plt.figure(figsize=(20,20))

# mask out DO_NOT_USE values of 1
masked_rate_im = np.ma.masked_where((rate_im.dq & dqflags.pixel['DO_NOT_USE'] > 0), rate_im.data)

cmap = matplotlib.cm.get_cmap("Greys").copy()  # Can be any colormap that you want after the cm
cmap.set_bad(color='blue') # color to mark all DO_NOT_USE pixels

plt.imshow(masked_rate_im, cmap=cmap, origin='lower', vmin=500,vmax=580.0)
plt.colorbar()
plt.show()

## Run calwebb_detector2

Run the output of detector1 (rate files) through calwebb_detector2 to obtain a set of calibrated files (cal files).


In [ ]:
# Run Calwebb_image2 on output files from detector1    
    
print('There are ', len(ratefiles), ' images.')
    
callist = []

# cycle through files
for im in ratefiles:

    calfile = Image2Pipeline.call(im, save_results=True)

    callist.append(calfile)

print(callist)

## Create and subtract mean sky background image

The input for the following steps is the list of calibrated files that were the result of calwebb_image2 processing. These files will be used to create the mean (or median) background image. Once the background image is created, it will be subtracted from each of the individual cal files, creating a set of background subtracted calibrated files. These files can then be processed through the last stage of the pipeline, calwebb_image3, to combine the background subtracted files into a dither combined mosaic image.

In [ ]:
# Set the filter being used in the data, so that it can be used in output file names.

filter = 'F2100W' 

In [ ]:
# Gather the list of cal files for the sky subtraction steps.

#miri_cal_files = glob.glob('*cal.fits') # this will collect all cal files in the directory

miri_cal_files = [ele.replace('rate', 'cal') for ele in ratefiles]

print(miri_cal_files)

In [ ]:
# Use the script make_sky to make and subtract off the sky background image.

simage = make_sky(miri_cal_files, scalebkg=scalebkg,
                  ds9regions=ds9region, exclude_above=exclude_above, exclude_delta=exclude_delta)

### Look at a sample image before and after sky subtraction

Also look at the mean sky image itself and write it out.

In [ ]:
# Look at cal image before sky subtraction

im = ImageModel(miri_cal_files[0])

plt.figure(figsize=(20,20))
plt.imshow(im.data, cmap='Greys', origin='lower', vmin=200,vmax=230)
plt.colorbar()
plt.show()

In [ ]:
# Look at the created median sky image and write to a file

drange_cal = [40., 230.]
show_image(simage, drange_cal[0], drange_cal[1], dmap=dmap)
fits.writeto(filter+'_sky.fits', simage, overwrite=True)

In [ ]:
# Gather sky subtracted cal files

miri_skysub_files = [ele.replace('cal', 'skysub_cal') for ele in miri_cal_files]

print(miri_skysub_files)

In [ ]:
# Look at cal image after sky subtraction

im2 = ImageModel(miri_skysub_files[0])

plt.figure(figsize=(20,20))
plt.imshow(im2.data, cmap='Greys', origin='lower', vmin=0,vmax=1)
plt.colorbar()
plt.show()

### Run Calwebb_image3 on background subtracted cal files

First create a new association file for the background subtracted files, and then use that file to process the set through calwebb_image3.


In [ ]:
# use asn_from_list to create association table
miri_asn_name = 'miri_'+filter+'_stage3_asn_skysub' # name of output asn file

asn = asn_from_list.asn_from_list(miri_skysub_files, rule=DMS_Level3_Base, product_name=miri_asn_name)

# dump association table to a .json file for use in image3

miri_asn_file = miri_asn_name+'.json'
with open(miri_asn_file, 'w') as outfile:
    outfile.write(asn.dump()[1])

In [ ]:
# Run calwebb_image3 (or Image3Pipeline) on sky subtracted data using association table.

cfg = dict() 

cfg['tweakreg'] = {}
cfg['tweakreg']['abs_refcat'] = 'GAIADR3'
#cfg['skymatch'] = {'skip' : True} 

cfg['resample']={}  # set up empty dictionary for multiple parameters to be set per step
cfg['resample']['rotation'] = rotation
cfg['resample']['kernel'] = 'gaussian'
cfg['resample']['weight_type'] = 'exptime'

#cfg['outlier_detection'] = {'save_intermediate_results' : True}  # Can set single parameters with this syntax
#cfg['source_catalog'] = {'deblend': True} # test to see if this crashes. If scikit-image is not installed, this will crash.
                     
output = Image3Pipeline.call(miri_asn_file, steps=cfg, save_results=True)

#### Look at the output combined mosaic and overlay the source catalog to see what sources were found

In [ ]:
# Look at the final i2d image (combined mosaic)

# Read your mosaic image into an ImageModel datamodel
miri_mosaic_file =  miri_asn_name + '_i2d.fits'
miri_mosaic = ImageModel(miri_mosaic_file)

plt.figure(figsize=(20,20))

fig, ax = plt.subplots(figsize=(20,20))

# Set up image
cax = ax.imshow(miri_mosaic.data, cmap='Greys', origin='lower', vmin=0,vmax=2)

# Set up colorbar
cb = fig.colorbar(cax)
cb.ax.set_ylabel('MJy/str',fontsize=14)

#Set labels 
ax.set_xlabel('X',fontsize=16)
ax.set_ylabel('Y',fontsize=16)
ax.set_title('Final MIRI mosaic', fontsize=16)
plt.tight_layout()

In [ ]:
print('Total exposure time of combined mosaic: ',miri_mosaic.meta.resample.product_exposure_time)

In [ ]:
# Compare to display of background subtracted i2d file to non-background subtracted file

# read in non-background subtracted file
default_pipeline_mosaic = ImageModel(files[4])

# Set up two plots side by side
fig, ax = plt.subplots(2, 1,figsize=[15,15])
cmap='Greys'

ax[0].set_title('Backround subtracted i2d file')
ax[1].set_title('Non-background subtracted i2d file')

cset1 = ax[0].imshow(miri_mosaic.data,cmap=cmap, origin='lower', vmin=0, vmax=2)
ax[0].set_xlabel('X',fontsize=14)
ax[0].set_ylabel('Y',fontsize=14)
cb1 = fig.colorbar(cset1, ax=ax[0])
cb1.ax.set_ylabel('MJy/str',fontsize=14)


cset2 = ax[1].imshow(default_pipeline_mosaic.data,cmap=cmap, origin='lower', vmin=214, vmax=220)
ax[1].set_xlabel('X',fontsize=14)
ax[1].set_ylabel('Y',fontsize=14)
cb2 = fig.colorbar(cset2, ax=ax[1])
cb2.ax.set_ylabel('MJy/str',fontsize=14)
plt.tight_layout()

In [ ]:
# Look at mosaic data and sources found with source_catalog

miri_catalog_file = miri_asn_name + '_cat.ecsv'

# Read in the source catalog
miri_source_cat = ascii.read(miri_catalog_file)

# Show the catalog sources on the mosaic
overlay_catalog(miri_mosaic.data, miri_source_cat, flux_limit=5e-7, vmin=0, vmax=2,
                title='Final MIRI mosaic with source catalog', dmap="Greys")